In [3]:
import os
import random
import timm

import torch
import torchvision
import torchvision.transforms as T
from torch.nn import functional as F

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch.nn as nn

from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, ConcatDataset

from PIL import Image
from tqdm import tqdm

import cv2
from IPython import display

import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

### CONFIG

In [12]:
class CONFIG:

    @staticmethod
    def set_seed(SEED):
        os.environ['PYTHONHASHSEED'] = str(SEED)
        random.seed(SEED)
        np.random.seed(SEED)
        torch.manual_seed(SEED)
        torch.cuda.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)
        torch.backends.cudnn.benchmark = True
    
    @staticmethod
    def set_path(root_path):
        train_path = f'{root_path}/train/'
        test_path = f'{root_path}/test/'

        return root_path, train_path, test_path

In [13]:
CONFIG.set_seed(0xC0FFE)
root_path, train_path, test_path = CONFIG.set_path('/root/Project/data_copy/data')

In [14]:
train_file = pd.read_csv(f'{root_path}/train2.csv')
train_file.head(3)

,ID,target
0,002f99746285dfdd.jpg,16
1,008ccd231e1fea5d.jpg,10
2,008f5911bfda7695.jpg,10


### Albumentations (Flip, Rotation, Resize(padding))

In [ ]:
def Flip(df:pd.DataFrame):

    # 생성할 디렉토리 이름
    directory_names = [    
        'Flip_Vertical', 
        'Flip_Horizontal',
    ]

    # 수행할 transforms
    transforms = [
        A.VerticalFlip(always_apply=True), 
        A.HorizontalFlip(always_apply=True), 
    ]
    
    # 디렉토리 생성
    for name in directory_names:
        path = f'{root_path}/{name}/'

        if not os.path.exists(path):
            os.makedirs(path)

    buffer_vertical = list()
    buffer_horizontal = list()

    for file_name, target in tqdm(df.values):

        # 이미지 읽어오기
        image_path = f'{root_path}/train/{file_name}'
        image = cv2.imread(image_path)

        for index in range(2):
            # Flip 수행
            fliped_image = transforms[index](image=image)["image"]

            # 해당 폴더에 저장
            f = file_name.split('.')[0]
            cv2.imwrite(f'{root_path}/{directory_names[index]}/{f}_{directory_names[index]}.jpg', fliped_image)

            if index == 0:
                buffer_vertical.append([f'{f}_{directory_names[index]}.jpg', target])
            else:
                buffer_horizontal.append([f'{f}_{directory_names[index]}.jpg', target])
    
    buffer_vertical = pd.DataFrame(buffer_vertical, columns=['ID', 'target'])
    buffer_horizontal = pd.DataFrame(buffer_horizontal, columns=['ID', 'target'])

    # 각각 train_flip_vertical.csv, train_flip_horizontal.csv로 저장
    buffer_vertical.to_csv(f'{root_path}/train_flip_vertical.csv', index=False)
    buffer_horizontal.to_csv(f'{root_path}/train_flip_horizontal.csv', index=False)

    return (buffer_vertical, buffer_horizontal)

Flip(train_file)
        

In [45]:
def Rotate(train_csv, vertical_csv, horizontal_csv):
    
    directory_names = [
        'Original_Rotation',
        'Flip_Vertical_Rotation', 
        'Flip_Horizontal_Rotation',
    ]
    for name in directory_names:
        path = f'{root_path}/{name}/'

        if not os.path.exists(path):
            os.makedirs(path)

    # key: 디렉토리 이름 / value: 이미지 csv파일
    files = {'train': train_csv, 
                'Flip_Vertical': vertical_csv,
                'Flip_Horizontal': horizontal_csv,}

    for i in range(1, 12):
        
        # 30도 간격으로 transform
        angle = i * 30
        transforms = A.Rotate(always_apply=True, limit=(angle, angle),
            interpolation=4, border_mode=0, value=(255, 255, 255))

        for index, key in enumerate(files):
            buffer = list()

            # 하위 디렉토리 만들기
            path = f'{root_path}/{directory_names[index]}/{directory_names[index]}_{angle}'
            if not os.path.exists(path):
                os.makedirs(path)

            # csv 파일에서 이미지 불러오기
            for file_name, target in tqdm(files[key].values):
                image_path = f'{root_path}/{key}/{file_name}'
                image = cv2.imread(image_path)

                # 이미지 회전
                rotated_image = transforms(image=image)["image"]

                # 해당 폴더에 저장
                f = file_name.split('.')[0]
                cv2.imwrite(f'{path}/{f}_Rotation_{angle}.jpg', rotated_image)

                buffer.append([f'{f}_Rotation_{angle}.jpg', target])
    
            buffer = pd.DataFrame(buffer, columns=['ID', 'target'])
            buffer.to_csv(f'{root_path}/{key}_rotation_{angle}.csv', index=False)

In [46]:
# Flip된 csv파일 불러오기
vertical_csv = pd.read_csv(f'{root_path}/train_flip_vertical.csv')
horizontal_csv = pd.read_csv(f'{root_path}/train_flip_horizontal.csv')

In [47]:
Rotate(train_file[:100], vertical_csv[:100], horizontal_csv[:100])

100%|██████████| 100/100 [00:01<00:00, 76.65it/s]
